In [8]:
%run Imports.ipynb
%run Functions.ipynb

You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' c

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/leftclick/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


all imports complete
Functions Imported


## Helper Functions

In [9]:
# Function that takes a query Id in the form Aftertaste/Distinguishing%20aftertaste%20and%20flavor
# Returns aftertaste distinguishing aftertaste flavor
# Removes slashes, %20, and lower cases, as well as removing stopwords.
def formatQuery(queryId,stem):
    queryIdWords = [x.lower() for x in queryId.replace("/"," ").replace("%20"," ").split(" ") if x.lower() not in stopWords]
    if(stem == True):
        return ' '.join(set(stemmer.stem(word) for word in queryIdWords)).strip()
    else:
        return ' '.join(list(dict.fromkeys(word for word in queryIdWords))).strip()

In [10]:
# Function that takes the paragraph associated with a query
# Returns the unique words without stopwords
def formatQueryParagraph(queryText,stem):
    queryParagraph = [x.strip() for x in queryText.split(" ") if x not in stopWords]
    if(stem):
        return ' '.join(set(stemmer.stem(word) for word in queryParagraph))
    return ' '.join(set(word for word in queryParagraph))



In [31]:
# testParagraphFile = "../Galago/data/paragraphCorpus.txt"
# sdmQueryFile = "../Galago/Queries/sdm/rih-query.json"

# QrelMap is a map of qrel ID to 
def loadData (qrelMap, qrelTestMap,queryText,splitLine):
#     qrelText = "../Java_API/Car-Grpc/src/main/resources/data/General/qrelMap-test.txt"
    qrelText = "../Java_API/Car-Grpc/src/main/resources/data/qrelMap-train.txt"

#     queryFile = "../Java_API/Car-Grpc/src/main/resources/Queries/Glove/rih-0-combine-query.json"
    paragraphFile = "paragraphCorpus.txt"
    
    with open(paragraphFile,'r') as f:
        for line in f:
            splitLine.append(list(filter(None,' '.join(line.split("__PERIOD__")).split(" "))))

#     with open(queryFile,'r') as f:
#         p = re.compile("\((.*?) \)")
        
#         for line in f:
#             queryText += ' '.join([x.strip() for x in p.findall(line)])

    with open(qrelText,'r') as f:
        for line in f:
            test = line.split(":")
            
            key = formatQuery(test[1],False).strip()
    
            value = formatQueryParagraph(test[2],False).strip()
            if(key not in qrelMap):
                qrelMap[key] = [value]
            else:
                qrelMap[key].append(value)
#     with open(qrelTestText,'r') as f:
#         for line in f:
#             test = line.split(":")
            
#             key = formatQuery(test[1],False).strip()
    
#             value = formatQueryParagraph(test[2],False).strip()
#             if(key not in qrelTestMap):
#                 qrelTestMap[key] = [value]
#             else:
#                 qrelTestMap[key].append(value)   

    print("loading complete")
    return queryText

In [32]:
qrelMap = dict()
qrelTestMap = dict()
queryText = ""
splitLine = []
queryText = loadData(qrelMap,qrelTestMap,queryText,splitLine)

loading complete


In [33]:
# Returns the weighted average embedding based on tf-idf calculation
# Gives a better representation of the average embedding
def tfidfCalc(testQuery,paragraphs):
    test = 0
    embeddings = []
    embeddingVocab = glove_model.wv.vocab
#     formattedQuery = formatQuery(testQuery,False)
    tfMap = {word: 0 for word in testQuery.split(" ")}
    dfMap = {word: 0 for word in testQuery.split(" ")}

    for paragraph in paragraphs:
        for word in testQuery.split(" "):
            if(word in paragraph):
                dfMap[word] += 1
                tfMap[word] += paragraph.count(word)
    for key in dfMap:
        tfMap[key] = tfMap[key]/len(paragraphs)
        if(dfMap[key] == 0):
            dfMap[key] = 0.001
        dfMap[key] = np.log(len(paragraphs)/dfMap[key])
        test = tfMap[key] * dfMap[key]
        if key in embeddingVocab:
            embeddings.append(glove_model[key] * test)
    
    averageEmbedding = np.sum(embeddings,axis = 0)
    return averageEmbedding
    


In [34]:
# Function to create map of word to top n simiar words.
# This uses the embedding of each word to find the most similar words
# Returns a string mapping the top n words related to each word in the query.
# Removes any words that exist in the original query
def independentN(queryWords,stem,n):
    independentSimilar = ""
    independentSimilarList = []
    queryWords = queryWords.split(" ")
#     print(queryWords)
    vocab = glove_model.wv.vocab
#     print([word for word in queryWords])
#     print([word in vocab for word in queryWords])
    similarMap = [(word,glove_model.most_similar(positive=[word],topn=n)) for word in queryWords if word in vocab]
    for key,value in similarMap:
        if(key in vocab and re.sub('[^a-zA-Z]+',"",key) != ""):
            if(stem):
                independentSimilar = re.sub('[^a-zA-Z]+',"",key) +':' + ' '.join(set(stemmer.stem(x[0]) for x in value if x[0] not in queryWords))
            else:
                independentSimilar = re.sub('[^a-zA-Z]+',"",key) +':' + ' '.join([re.sub('[^a-zA-Z]+',"",x[0])+ "+" + str(round(x[1],4)) for x in value if x[0] not in queryWords and re.sub('[^a-zA-Z]+',"",x[0]) != ""])
            independentSimilarList.append(independentSimilar)
        else:
            continue
#     print(independentSimilarList)
    return independentSimilarList

In [36]:
# independentN(testQuery,False,50)

In [37]:
# Function to compare average embedding to each retrieved word 
# Takes a queryId
# This is very rough, as it takes the average of the entire topN
def independentRelevance(queryId):
    relevantExpansions = []
    relevantExpansionWords = ""
    queryEmbedding = tfidfCalc(queryId,qrelMap[queryId])

    independentEmbedding = independentN(queryId,False,10)
    for iE in independentEmbedding:
        independentEmbedding = averageEmbedding(' '.join(set(iE.split(":")[1].split(" "))))
#         print(iE.split(":")[0])
#         print(abs(np.sum(queryEmbedding - independentEmbedding)))
        if(abs(np.sum(queryEmbedding - independentEmbedding)) < 10):
            relevantExpansions.append(iE)
            relevantExpansionWords += (iE.split(":")[0]) + " "
    return [relevantExpansionWords.strip(),relevantExpansions]

In [38]:
# Function to calculate the average embedding value of a query
# Takes a query
def averageEmbedding(queryId):
    embeddings = []
    queryList =queryId.split(" ")
    embeddingVocab = glove_model.wv.vocab
    for queryWord in queryList:
        if queryWord in embeddingVocab:
            embeddings.append(glove_model[queryWord.strip()])
        averageEmbedding = np.sum(embeddings,axis = 0)
    return averageEmbedding



# Function to create list of the top N similar words
# This uses the average embedding of the entire query
# Returns a list of the top n words, removing any words that exist in the original query.
def averageN(queryId,stem, n):
    avgE = tfidfCalc(queryId,qrelMap[queryId])
    if(avgE.size == 300):
        if(stem):
            return ' '.join(queryId.split(" ")) + ':' + ' '.join(set(stemmer.stem(x[0]) for x in glove_model.similar_by_vector(avgE,topn=n) if x[0] not in queryId))
        return ' '.join(queryId.split(" ")) + ':' + ' '.join(set(x[0] for x in glove_model.similar_by_vector(avgE,topn=n) if x[0] not in queryId))



# Function to create list of the top N similar words and their similarity measures
# This uses the average embedding of the entire query
# Returns a list of the top n words, removing any words that exist in the original query.
def averageNScores(queryWord,stem, n,k):
    queryExpansionWords =[]
    queryWordEmbedding = glove_model[queryWord]
    similarWords = glove_model.similar_by_vector(queryWordEmbedding,topn=n)
    similarWords = [list(x) for x in similarWords if x[0] != queryWord]
    
    for i in range(5):
        mostSimilar = similarWords.pop(0)
        queryExpansionWords.append(mostSimilar)
        for x in similarWords:
            x[1] = glove_model.similarity(x[0],mostSimilar[0]) 
        similarWords = sorted(similarWords, key=lambda x: x[1],reverse=True)
#         print("SORTED" + str(similarWords))
        similarWords = similarWords[:len(similarWords)-k]
    return queryExpansionWords


In [39]:
# For each query word, check each independent embedding word
# If the embedding word is not relevant enough to the average of the query embedding disregard it.
# Return a string containing all relevant words for each query word
# Filters based on the average query embedding using tfidf
def independentRelevanceFine(queryId,paragraphs):
    relevantExpansions = ""
    queryEmbedding = tfidfCalc(queryId,paragraphs)
    independentEmbedding = independentN(queryId,False,10)
    outputEmbeddings = dict()
    for iE in independentEmbedding:
        queryWord = iE.split(":")[0]
        independentEList = iE.split(":")[1].split(" ")
        relevantExpansionWords = []
        for word in independentEList:
            if word in glove_model.wv.vocab:
                wordEmbedding = glove_model[word]
                if(abs(np.sum(queryEmbedding - wordEmbedding))<15):
                    relevantExpansionWords.append(word + " ")
        if(relevantExpansionWords != []):
            outputEmbeddings[queryWord] = relevantExpansionWords
            
    return outputEmbeddings


In [40]:
# For each query word, check each independent embedding word
# If the embedding word is not relevant enough to the average of the query embedding disregard it.
# Return a string containing all relevant words for each query word
# Filters based on the average query embedding using tfidf
def independentRelevanceN(queryId,paragraphs,n):
    relevantExpansions = ""
    queryEmbedding = tfidfCalc(queryId,paragraphs)
    independentEmbedding = independentN(queryId, False, n)
    outputEmbeddings = ""
    
    for iE in independentEmbedding:
        queryWord = iE.split(":")[0]
        independentEList = iE.split(":")[1].split(" ")
        relevantExpansionWords = " "
        for word in independentEList:
            if word in glove_model.wv.vocab:

                wordEmbedding = glove_model[word]
                if(abs(np.sum(queryEmbedding - wordEmbedding))<25):
                    relevantExpansionWords += word + " "
        
        if(relevantExpansionWords != []):
            relevantExpansionWords = ' '.join(sorted(relevantExpansionWords.split(" ")))
            outputEmbeddings += queryWord + ":" + relevantExpansionWords + ";"
#         else:
#             print(word)
            
    return outputEmbeddings


In [41]:
# Function which checks the ratio of top n similar words that actually appear
# in the paragraph associated with the query in the qrel file.
# Takes the paragraphs and the queryId and the number of similar words to search for
def matchedAverage(qrelParas, queryId,n):
    for paragraph in set(qrelParas):
        matched = averageN(queryId,False,n).split(":")[1].split(" ")
        matchedWords = set([x for x in matched if x in paragraph])
        if(matchedWords == set()):
            return ["",0]
        else:
            return [list(matchedWords),len(matchedWords)/len(matched)]

In [42]:
# Function which checks the ratio of top n similar words that actually appear
# in the paragraph associated with the query in the qrel file.
# Takes the paragraphs and the queryId and the number of similar words to search for
def matchedIndependent(qrelParas,queryId,n):
    
    allMatched = []
    for paragraph in set(qrelParas):
        matchedList = independentN(queryId,False,n)
        for word in matchedList:
            matched = word.split(":")[1].split(" ")
            matchedWords = set(x for x in matched if x in paragraph)
            if(matchedWords != set()):
                allMatched.append(list(matchedWords))
    flatList = set(x for y in allMatched for x in y)

    return [list(flatList),len(flatList)/len(matched)]

In [43]:
testQuery = formatQuery("Allergy/Cause/Other%20environmental%20factors ",False)

testQueryList = list(islice(qrelMap, 5))

independentN(testQuery,False,10)

['allergy:allergies+0.7246 allergic+0.6522 asthma+0.6489 fauci+0.5374 sufferers+0.5139 arthritis+0.5057 allergens+0.5012 medication+0.4827 medications+0.4824 anaphylaxis+0.4758',
 'cause:causes+0.8481 caused+0.7603 causing+0.7388 damage+0.595 serious+0.5714 severe+0.5707 result+0.5643 harm+0.5536 might+0.5496 likely+0.5461',
 'environmental:ecological+0.6484 pollution+0.6385 conservation+0.6369 environment+0.6296 protection+0.5658 environmentalists+0.5566 epa+0.5496 safety+0.5473 sustainability+0.5444 climate+0.5329',
 'factors:factor+0.7534 determining+0.5523 affect+0.5515 reasons+0.5485 considerations+0.5426 mitigating+0.536 impact+0.5131 particular+0.5085 increases+0.5082 affecting+0.5037']

In [44]:
class TestStringMethods(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        warnings.simplefilter("ignore")
        cls.testString = "immigration federal agriprocessors raid controversies:enforcement investigations illegal laws fbi investigation"
        cls.testQuery = "agriprocessors controversies federal immigration raid"
        
    def test_format_query(self):
        self.assertEqual(sorted(formatQuery("Agriprocessors/Controversies/Federal%20immigration%20raid",False).split(" ")),sorted(self.testQuery.split(" ")))

    def test_average_embedding(self):
        self.assertEqual(len(tfidfCalc(self.testQuery,qrelMap[self.testQuery])), glove_model.vector_size)

    def test_average_embedding_length(self):
        self.assertTrue(averageN(testQuery,False,10),self.testString)
        self.assertFalse('Foo'.isupper())

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.035s

OK


## Independent Embedding Generation

In [45]:
print("Writing out independent embedding map")
topANarr = []
allWords = []
with open("../Java_API/Car-Grpc/src/main/resources/data/topIN/independent-Glove-train.txt",'w') as topIN:
    topINarr = []
    for x in qrelMap:        
        uniqueQueryWords = formatQuery(x,False)
        allWords.extend([x for x in uniqueQueryWords.split(" ") if x not in allWords and x.lower() not in stopWords])
    for word in allWords:
        expansionTerms = independentN(word, False, 1000)
        if(expansionTerms != []):
            topINarr.extend(expansionTerms)
    topIN.write(';'.join(topINarr))
    print("Independent embedding map written to topIN.txt")
print("finished")

Writing out independent embedding map
Independent embedding map written to topIN.txt
finished


In [ ]:
print("Writing out independent embedding map")
topANarr = []
with open("../Java_API/Car-Grpc/src/main/resources/data/StopWords") as sWordsFile:
    stopWords = sWordsFile.read().split("\n")
allWords = []
with open("../Java_API/Car-Grpc/src/main/resources/data/topIN/topIN-test.txt",'w') as topIN:
    topINarr = []
    for x in qrelMap:        
        uniqueQueryWords = formatQuery(x,False)
        allWords.extend([x for x in uniqueQueryWords.split(" ") if x not in allWords and x not in stopWords])
    for word in allWords:
        expansion = independentN(word, False, 100)
        words = []
        if(expansion != []):
            e = expansion[0].split(":")[1].split(" ")
            for w in e:
                if(len(w) > 1 and w not in stopWords):
#                 if w in ' '.join(qrelMap[uniqueQueryWords]).split(" ") and len(w) > 1 and w not in stopWords: 
                    words.append(w)
            if words != []:
                output = [expansion[0].split(":")[0] +":" + ' '.join(words)]
                topINarr.extend(output)
    topIN.write(';'.join(topINarr))
    print("Independent embedding map written to topN.txt")
print("finished")

### ELMo STUFF

In [48]:
subsetQueriesList = []
with open("test-random-subset.txt") as subsetQueries:
    for line in subsetQueries:
        subsetQueriesList.append(line.replace("\n",""))
#     while (line = subsetQueries.readline()) != null:
#         print(subsetQueries.readline())

### Global Average ELMo Embeddings

In [49]:
# print(subsetQueriesList)

['enwiki:Ionic%20compound/Structure/Defects', 'enwiki:Antibiotics/Medical%20uses/Administration', 'enwiki:Instant%20coffee/Use', 'enwiki:Beach%20nourishment/Causes%20of%20erosion', 'enwiki:Coevolution/Coevolution%20outside%20biology', 'enwiki:Storm%20surge/Historic%20storm%20surges', 'enwiki:Disturbance%20(ecology)/Importance', 'enwiki:Anti-Slavery%20International/Overview', 'enwiki:Disgust/Morality/Political%20orientation', 'enwiki:World%20Health%20Organization/Controversies', 'enwiki:Onboarding/Recommendations%20for%20practitioners', 'enwiki:Drinking%20water/Regulation/European%20Union', 'enwiki:Ionic%20compound/Formation', 'enwiki:Mexican%20cuisine/Regional%20cuisines/Northern%20Mexico', 'enwiki:Purified%20water/Health%20effects%20of%20drinking%20purified%20water', 'enwiki:Elaeis%20guineensis/Disease', 'enwiki:Tsunami/Generation%20mechanisms/Man-made%20or%20triggered%20tsunamis', 'enwiki:Non-governmental%20organization/Critiques/Challenges%20to%20legitimacy', 'enwiki:Instant%20coffe

In [50]:
h5py_file = h5py.File("elmo-embeddings/elmo-embeddings-global-avg.hdf5", 'r')
avg_queries = []

with open("500kvocab.txt","r") as vocab_file: # Get all Vocab words and link this to the elmo embeddings
    vocab_list = vocab_file.read().split("\n")
    vocab_to_elmo = dict()

    for x in range(0,len(vocab_list)-1):     # For every word in the vocab, create the key value pair.
        vocab_to_elmo[vocab_list[x]] = h5py_file.get(str(x))
        
for q in subsetQueriesList:
    tokens = list(set(formatQuery(q, False).replace("enwiki:","").split(" ")))
    print(tokens)
    avg_tokens = []
    vectors = elmo.embed_sentence(tokens)
    avg_query = np.average(np.average(vectors,axis=0),axis=0)
    avg_queries.append(avg_query)

average_similarities = defaultdict(list)
y = 0
for averageQ in avg_queries:
    start = time.time()
    similarities = dict()
    for x in vocab_to_elmo:     # For every vocab word
        test = scipy.spatial.distance.cosine(vocab_to_elmo[x],averageQ)        # Store the cosine similarity to the average query in a submap
        similarities[x] = test
    average_similarities[subsetQueriesList[y]].append(similarities)    # Store the word and its similarity in a map of query to similarity.
    end = time.time() - start
    print(end)

#     print(end)
    y += 1
    
#Similarities is a map of vocab word to similarity to the average query.
# List with entry of all most similar words for each query word.

import operator
import re
with open("Global_ELMo/global_avg_similarity.txt","w") as f:
    for x in range(0,len(subsetQueriesList)):
        tokens = list(set(formatQuery(subsetQueriesList[x], False).replace("enwiki:","").split(" ")))
        sorted_d = sorted(average_similarities[subsetQueriesList[x]][0].items(), key=operator.itemgetter(1))
        f.write(str(subsetQueriesList[x]).replace("\n","") + "@" + str([x[0] +"+" + str(x[1]) for x in sorted_d[:150] if re.sub('[^a-zA-Z]+',"",x[0]) not in tokens]) + "\n")

['compound', 'defects', 'ionic', 'structure']
['administration', 'medical', 'uses', 'antibiotics']
['coffee', 'instant', 'use']
['beach', 'nourishment', 'erosion', 'causes']
['biology', 'coevolution', 'outside']
['storm', 'surges', 'historic', 'surge']
['(ecology)', 'disturbance', 'importance']
['overview', 'international', 'anti-slavery']
['orientation', 'disgust', 'morality', 'political']
['world', 'health', 'controversies', 'organization']
['practitioners', 'onboarding', 'recommendations']
['european', 'water', 'union', 'drinking', 'regulation']
['compound', 'formation', 'ionic']
['regional', 'cuisine', 'mexican', 'cuisines', 'northern', 'mexico']
['purified', 'water', 'drinking', 'health', 'effects']
['elaeis', 'disease', 'guineensis']
['generation', 'tsunami', 'tsunamis', 'mechanisms', 'man-made', 'triggered']
['non-governmental', 'challenges', 'organization', 'critiques', 'legitimacy']
['hazards', 'coffee', 'instant', 'health']
['olfactory', 'receptor', 'neuron', 'olfaction', 'sy

In [25]:
import operator
import re
with open("Global_ELMo/global_avg_similarity.txt","w") as f:
    for x in range(0,len(subsetQueriesList)):
        tokens = list(set(formatQuery(subsetQueriesList[x], False).replace("enwiki:","").split(" ")))
        sorted_d = sorted(average_similarities[subsetQueriesList[x]][0].items(), key=operator.itemgetter(1))
        f.write(str(subsetQueriesList[x]).replace("\n","") + "@" + str([x[0] +"+" + str(x[1]) for x in sorted_d[:1000] if re.sub('[^a-zA-Z]+',"",x[0]) not in tokens]) + "\n")

NameError: name 'average_similarities' is not defined

### Load independent and dependent local ELMo data

In [51]:
#Context dependent Local ELMo paragraphs and queries - from run file
# with open("../Java_API/Car-Grpc/src/main/resources/data/Local_ELMo/train-subset-localELMo.json", "r") as read_file:
#     data = json.load(read_file)
import json
#Context dependent Local ELMo paragraphs and queries - from run file
# with open("../Java_API/Car-Grpc/src/main/resources/data/Local_ELMo/test-subset-localELMo.json", "r") as read_file:
#     data = json.load(read_file)
    
#Context independent Local ELMo paragraphs and queries - from run file
with open("../Java_API/Car-Grpc/src/main/resources/data/Local_ELMo/train-subset-indy-localELMo.json", "r") as read_file:
    indy_data = json.load(read_file)

### Generates query-vocab map for context independent Local ELMo

In [52]:
# Generate context independent vocab of queries to list of most similar words with rankings, Local ELMo

indy_query_vocab_map = dict()
for query_num in range(50):
    vocab_set = set()
    for paragraph in indy_data.get("queries")[query_num].get("Paragraphs")[:20]:
        for w in paragraph.replace("[","").replace("]","").split(","):
            vocab_set.add(w)
    indy_query_vocab_map[indy_data.get("queries")[query_num].get("QueryId")] = vocab_set
    

### Generates word-similarity map for all queries

In [53]:
# Context independent Word-Similarity map for all queries in the train set 
import time
query_to_word_map = dict()

for query in indy_data.get("queries"): # For each query in the train set
    querySimilarityList = []
    querySimilarityMapList = []
    sortedParagraphs = []
    start = time.time()
    
    QueryId = query.get("QueryId")
    
    # Get the formatted Query Id
    queryFormatted = list(set(formatQuery(QueryId,False).replace("enwiki:","").split(" ")))
    
    # Get the average embedding for each word in the query.
    averageQueryEmbedding = np.average(elmo.embed_sentence(queryFormatted),axis=0)
#     print(averageQueryEmbedding.shape)

    # Average embedding of the entire query
    entireQueryEmbedding = np.average(averageQueryEmbedding,axis=0)
#     print(entireQueryEmbedding.shape)
    
    paragraphSimilarity = []
    word_to_similarity_map = dict()
    for word in indy_query_vocab_map[QueryId]: # For each word returned for that query
        averageWordEmbedding = np.average(elmo.embed_sentence([word.strip()]),axis=0)
        similarity = scipy.spatial.distance.cosine(entireQueryEmbedding,averageWordEmbedding) #Get the similarity
        word_to_similarity_map[word] = similarity
    query_to_word_map[QueryId] = word_to_similarity_map
    end = time.time() - start
    print(end)
# print(query_to_word_map)

#     import re

63.13508200645447
80.12757992744446
63.86967325210571
64.15473318099976
49.01329731941223
62.67295455932617
63.50104594230652
54.21759653091431
63.258044719696045
68.97375059127808
55.325474977493286
60.50427174568176
66.39211392402649
66.01664543151855
53.547096252441406
90.77966356277466
74.03459286689758
74.56365966796875
80.88692235946655
43.76298379898071
55.44410943984985
58.158263206481934
54.69401812553406
79.668710231781
63.40876078605652
72.80970788002014
74.92783665657043
79.57811617851257
76.46160531044006
54.99807000160217
61.25098514556885
88.28884816169739
84.49814462661743
56.924975872039795
66.39566206932068
78.55631828308105
83.72816276550293
95.51873350143433
61.53771209716797
70.40596842765808
66.3760027885437
89.37235474586487
70.33120965957642
74.26629900932312
71.4531500339508
72.5851879119873
69.4743435382843
44.398423194885254
53.65299367904663
81.54960703849792


### File output for context independent Local ELMo

In [54]:
# Write out context independent weights to file for local ELMo

import operator
with open("../Java_API/Car-Grpc/src/main/resources/data/Local_ELMo/noContextWeightsTest.txt", "w") as f:
    for q in query_to_word_map: # For each query word store the list, sorted the top n by that query word

        sorted_x = sorted(query_to_word_map[q].items(), key=operator.itemgetter(1))

        f.write(q + "@" + str([re.sub('[^0-9a-zA-Z]+',"",x[0]) +"+"+ str(x[1]) for x in sorted_x if x[0] not in q][:10000]) + "\n")


### Context Dependent Local ELMo model

In [47]:
# This block generates a list of the top n words most similar to each query word.
# This is done by calculating the similarity between each word in the top 1000 paragraphs and the query words.
# This is stored in a large list of maps {'qWord1': 0.256, 'qWord2': 0.814, 'qWord3': 0.783, ...}
# It is sorted by each of these query words and the top most similar words are taken and added to a separate list.
# This forms the output which is a list of length q containing the top n expansion words where q is the number of 
# query terms.

wa = True
for query in data.get("queries"): # For each query in the train set
    querySimilarityList = []
    querySimilarityMapList = []
    sortedParagraphs = []
    start = time.time()
    
    # Get the formatted Query Id
    queryFormatted = list(set(formatQuery(query.get("QueryId"),False).replace("enwiki:","").split(" ")))
    
    # Get the average embedding for each word in the query.
    averageQueryEmbedding = np.average(elmo.embed_sentence(queryFormatted),axis=0)
#     print(averageQueryEmbedding.shape)
    entireQueryEmbedding = np.average(averageQueryEmbedding,axis=0)
    
    paragraphEmbeddingList = []
    paragraphSimilarity = []
    
    for paragraph in query.get("Paragraphs")[:20]: # For each paragraph returned for that query
        sentenceList = []
        sentenceSimilarity = []
        
        paragraph = paragraph.replace("[","").replace("]","").split(",") # format paragraph
        sentences = list(filter(None,''.join(paragraph).split("fullstop")))

        for x in range(0,len(sentences)): # For each sentence, make it a list and add it back.
            cleanedSentence = sentences[x].strip().split(" ")
            sentences[x] = cleanedSentence
            
        sentencesEmbeddings = elmo.embed_batch(sentences) # Run elmo on the list of sentences in a paragraph

        z = 0
        
        for sentenceEmbedding in sentencesEmbeddings: #For each sentence in the paragraph
            sentence = sentences[z]
            y = 0

            perWordEmbedding = np.average(sentenceEmbedding, axis=0) # Average out each word embedding

            for wordEmbed in perWordEmbedding: # For word in sentence
                paraWordSimilarity = dict()
                querySimilarityMap = dict()
                x = 0
            
                paraWordSimilarity["sentenceWord"] = sentence[y] # Store a reference to the word
                similarity = round(scipy.spatial.distance.cosine(wordEmbed,entireQueryEmbedding),3) #Get the similarity
                paraWordSimilarity["entireQuery"] = similarity

                for qWordEmbed in averageQueryEmbedding: # For query word
            
                    similarity = round(scipy.spatial.distance.cosine(wordEmbed,qWordEmbed),3) #Get the similarity
            
                    paraWordSimilarity[queryFormatted[x]] = similarity # Store the similarity mapped to query word
                    x += 1
                    
                if(wa): # Weighted Average Flag
                    if not any(d.get("sentenceWord",None) == sentence[y] for d in querySimilarityMapList):
                        x = 0
                        querySimilarityMap["sentenceWord"] = sentence[y] # Store a reference to the word
                        similarity = round(scipy.spatial.distance.cosine(wordEmbed,entireQueryEmbedding),3) #Get the similarity
                        querySimilarityMap["entireQuery"] = similarity

                        for qWordEmbed in averageQueryEmbedding:
                            similarity = round(scipy.spatial.distance.cosine(wordEmbed,qWordEmbed),3) #Get the similarity

                            querySimilarityMap[queryFormatted[x]] = [similarity]
                            x += 1
                        querySimilarityMapList.append(querySimilarityMap)
                    else:
                        for m in range(0,len(querySimilarityMapList)-1):
                            if querySimilarityMapList[m]["sentenceWord"] == sentence[y]:
                                x = 0
                                for qWordEmbed in averageQueryEmbedding:
                                    similarity = round(scipy.spatial.distance.cosine(wordEmbed,qWordEmbed),3) #Get the similarity
                                    querySimilarityMapList[m].get(queryFormatted[x]).append(similarity)
                                    x += 1
                else:
                    querySimilarityList.append(paraWordSimilarity) # Store all similarities for the query
                y += 1
            z += 1
    
    end_time = time.time() - start
    print(end_time)
    import re

    
    queryFormatted.append("entireQuery")
    for k in queryFormatted: # For each query word store the list, sorted the top n by that query word
        testList = []

        if(wa):
            for d in querySimilarityMapList:
#                 queryFormatted = queryFormatted.append("entireQuery")
                for q in queryFormatted:
                    d[q] = round(np.average(d[q]),3) # Choose top N occurrences by sorting and slicing
            testList = sorted(querySimilarityMapList, key=lambda x: (x[k]))
#             print(re.sub('[^0-9a-zA-Z]+',"",testList[0].get("sentenceWord")))
            if(k == ":"):
                k == "colon"
            if(k == "entireQuery"):
                expansionWords = k + ":" + ' '.join(set([re.sub('[^0-9a-zA-Z]+',"",x.get("sentenceWord")) +"+"+ str(x.get(k)) for x in testList if x.get("sentenceWord") != k]))
            else:
                expansionWords = k + ":" + ' '.join(set([re.sub('[^0-9a-zA-Z]+',"",x.get("sentenceWord")) +"+"+ str(x.get(k)) for x in testList if x.get("sentenceWord") != k]))

        else:
            testList = sorted(querySimilarityList, key=lambda x: (x[k]))
            if(k == "entireQuery"):
                expansionWords = k + ":" + ' '.join(set([re.sub('[^0-9a-zA-Z]+',"",x.get("sentenceWord")) +"+"+ str(x.get(k)) for x in testList if x.get("sentenceWord") != k]))
            else:
                expansionWords = k + ":" + ' '.join(set([re.sub('[^0-9a-zA-Z]+',"",x.get("sentenceWord")) +"+"+ str(x.get(k)) for x in testList if x.get("sentenceWord") != k]))

        sortedParagraphs.append(expansionWords)
    if wa:
        with open("../Java_API/Car-Grpc/src/main/resources/data/Local_ELMo/embeddingsAverageWeightsUnlimitedTest.txt", "a") as e:
            e.write(query.get("QueryId") + "@" + str(sortedParagraphs) + "\n")
    else:
        with open("../Java_API/Car-Grpc/src/main/resources/data/Local_ELMo/embeddingsGreedyWeights.txt", "a") as e:
            e.write(query.get("QueryId") + "@" + str(sortedParagraphs) + "\n")    


14.130402088165283
14.677674055099487
11.640497922897339
11.794345140457153
13.075102806091309
13.187154293060303
11.370903491973877
13.11812710762024
12.502481460571289
24.354890823364258
14.55323314666748
13.51351547241211
13.202198028564453
14.649916887283325
12.728102922439575
11.496967792510986
11.198596239089966
15.023398160934448
11.994405508041382
14.722891330718994
13.716572523117065
11.847083806991577
8.657777309417725
12.111258268356323
12.858834981918335
16.836432695388794
16.407613039016724
12.931836128234863
15.86539912223816
13.054662704467773
11.70007610321045
12.831973791122437
12.676647663116455
12.360377073287964
12.368247509002686
12.524710655212402
15.410269021987915
14.853639125823975
12.124898910522461
14.844342947006226
12.67747974395752
14.718013286590576
13.601699829101562
15.923472166061401
11.284835577011108
14.526308059692383
13.78287935256958
12.07865571975708
15.51027536392212
15.803512334823608


### Helpful Code maybe?

In [ ]:
with open("../Java_API/Car-Grpc/src/main/resources/Queries/ELMo/test-random-subset.txt","w") as test_queries:
    querySample = data.get("queries")
    sample50test = random.sample(querySample,50)
    obj = json.dumps({'queries': sample50test})
    json.dump(obj,test_queries)

In [ ]:
# average_similarities = defaultdict(list)
# y = 0
# for averageQ in avg_queries[:1]:
#     similarities = dict()
#     # For every vocab word
#     for x in vocab_to_elmo:
#         # Store the cosine similarity to the average query in a submap
#         test = scipy.spatial.distance.cosine(vocab_to_elmo[x],averageQ)
#         similarities[x] = test
#     # Store the word and its similarity in a map of query to similarity.
#     average_similarities[subsetQueriesList[y]].append(similarities)
#     y += 1

# print(average_similarities[subsetQueriesList[0]])

In [ ]:
# with open("Global_ELMo/independentSimilarity.txt","w") as avgSim:
#     for x in range(len(subsetQueriesList)):
#         tokens = list(set(formatQuery(subsetQueriesList[x], False).replace("enwiki:","").split(" ")))
#         avgSim.write(subsetQueriesList[x] + "@" + str(sorted(average_similarities[x].items(), key=operator.itemgetter(1))) + "\n")
# import operator
# y = 0
# with open("Global_ELMo/independentSimilarity.txt","w") as avgSim:
#     for y in range(len(subsetQueriesList) -1):
#         for x in average_similarities[subsetQueriesList[y]]:
#             avgSim.write(subsetQueriesList[y] + "@" + str(sorted(x.items(), key=operator.itemgetter(1))[:50]))

    #     print(x)
    #     print(average_similarities[subsetQueriesList[42]].get(x))
# print(average_similarities[0])

In [ ]:
# # Function that takes a number of expansion words wanted per query word.
# def topNIndependentElmo(n):
#     with open("../Java_API/Car-Grpc/src/main/resources/data/topINELMo/topINEComplete.txt", "w") as ELMoFile:
#         outputList = ""
#         # For each query word, sort these and get the top N words.
#         for x in range(len(all_similarities)-1):
            
#             output = sorted(all_similarities[x].items(), key=lambda kv: kv[1])

#             topNWords = [y[0] for y in output[:n+1] if y[0] not in tokens]
#             outputList += ' '.join(topNWords) + "\n"
            

#         ELMoFile.write(outputList)

In [ ]:
# # This block generates a map of each unique vocab word to a list of all similarities
# # This allows the average of these embeddings to be taken.


# queryEmbeddingList = []
# querySimilarityList = []

# for query in data.get("queries")[:1]: # For each query in the train set
#     start = time.time()
    
#     # Get the formatted Query Id
#     queryFormatted = list(set(formatQuery(query.get("QueryId"),False).replace("enwiki:","").split(" ")))
    
#     # Get the average embedding for each word in the query.
#     averageQueryEmbedding = np.average(elmo.embed_sentence(queryFormatted),axis=0)
    
#     paragraphEmbeddingList = []
#     paragraphSimilarity = []
    
#     for paragraph in query.get("Paragraphs")[:500]: # For each paragraph returned for that query
#         sentenceList = []
#         sentenceSimilarity = []
        
#         paragraph = paragraph.replace("[","").replace("]","").split(",") # format paragraph
#         sentences = list(filter(None,''.join(paragraph).split("fullstop")))

#         for x in range(0,len(sentences)): # For each sentence, make it a list and add it back.
#             cleanedSentence = sentences[x].strip().split(" ")
#             sentences[x] = cleanedSentence
            
#         sentencesEmbeddings = elmo.embed_batch(sentences) # Run elmo on the list of sentences in a paragraph

#         z = 0
        
#         for sentenceEmbedding in sentencesEmbeddings: #For each sentence in the paragraph
#             sentence = sentences[z]
#             y = 0

#             perWordEmbedding = np.average(sentenceEmbedding, axis=0) # Average out each word embedding

#             for wordEmbed in perWordEmbedding: # For word in sentence
#                 paraWordSimilarity = dict()
#                 x = 0
            
#                 paraWordSimilarity["sentenceWord"] = sentence[y] # Store a reference to the word

#                 for qWordEmbed in averageQueryEmbedding: # For query word
            
#                     similarity = round(scipy.spatial.distance.cosine(wordEmbed,qWordEmbed),3) #Get the similarity
#                     paraWordSimilarity[queryFormatted[x]] = similarity # Store the similarity mapped to query word
#                     x += 1
                    
#                 querySimilarityList.append(paraWordSimilarity) # Store all similarities for the query
#                 y += 1
#             z += 1
    
#     end_time = time.time() - start
#     print(end_time)
    
#     sortedParagraphs = []
    
#     for k in queryFormatted: # For each query word store the list, sorted the top n by that query word
#         testList = []
#         testList = sorted(querySimilarityList, key=lambda x: (x[k]))
#         sortedParagraphs.append(testList[:10])
        
# with open("../Java_API/Car-Grpc/src/main/resources/data/Local_ELMo/embeddings3.txt", "w") as e:
#     e.write("{" + str(sortedParagraphs) + "}")

In [ ]:
# # Creates a list of queries, with a list of paragraphs, with a list of sentences, with a list of words
# # Each word has it's similarity measure to each query word. This is useful for looking at the output directly.
# # The next block of code is useful for actually using ELMo programmatically.
# # This block also contains the actual embedding values stored in a list of lists for each paragraph.


# # For each query in the train set
# queryEmbeddingList = []
# querySimilarityList = []

# for query in random.sample(data.get("queries"),500): 
#     print(query.get("QueryId"))
#     start = time.time()
    
#     # Get the formatted Query Id
#     queryFormatted = list(set(formatQuery(query.get("QueryId"),False).replace("enwiki:","").split(" ")))
#     print(queryFormatted)
#     # Get the average embedding for each word in the query.
#     averageQueryEmbedding = np.average(elmo.embed_sentence(queryFormatted),axis=0)
    
#     paragraphEmbeddingList = []
#     paragraphSimilarity = []
    
#     # For each paragraph returned for that query
#     for paragraph in query.get("Paragraphs")[:1]:
# #         print(paragraph)
#         # format it and give it to the elmo embedder.
#         paragraph = paragraph.replace("[","").replace("]","")
#         test = paragraph.split(",")
#         sentences = list(filter(None,''.join(test).split("fullstop")))
# #         print(sentences)
#         for x in range(0,len(sentences)):
#             cleanedSentence = sentences[x].strip().split(" ")
#             sentences[x] = cleanedSentence
# #         print(len(sentences[0]) + len(sentences[1]))
#         # Run elmo on the list of sentences in a paragraph
#         sentencesEmbeddings = elmo.embed_batch(sentences)
# #         print(len(sentencesEmbeddings[0][0]) + len(sentencesEmbeddings[1][0]))
#         sentenceList = []
#         sentenceSimilarity = []
#         z = 0
#         #For each sentence in the paragraph
#         for sentenceEmbedding in sentencesEmbeddings:
#             sentenceWordSimilarity = []
#             sentence = sentences[z]
# #             print(sentence)
#             # Average out each word embedding
#             perWordEmbedding = np.average(sentenceEmbedding, axis=0)
            
# #             print("No. of Similarities:              " + str(len(perWordEmbedding) * len(averageQueryEmbedding)))
#             y = 0
    
#             # For word in sentence
#             for wordEmbed in perWordEmbedding:
#                 word = sentence[y]

#                 sentenceWordSimilarity.append(word)
#                 paraWordSimilarity = dict()
#                 x = 0
#                 # For query word
#                 for qWordEmbed in averageQueryEmbedding:
                    
#                     #Get the similarity between them
#                     paraWordSimilarity[queryFormatted[x]] = round(scipy.spatial.distance.cosine(wordEmbed,qWordEmbed),3)
#                     x += 1
#                 sentenceWordSimilarity.append(paraWordSimilarity)    
#                 y += 1
# #             print("Number of similarities Generated: " + str(len(sentenceWordSimilarity) * len(paraWordSimilarity)))
#             sentenceSimilarity.append(sentenceWordSimilarity)
#             sentenceList.append(perWordEmbedding)
#             z += 1

#         # Create a list of the sentence embeddings  and similarities
#         paragraphEmbeddingList.append(sentenceList)
#         paragraphSimilarity.append(sentenceSimilarity)
        
#     # Create a list of the embeddings and similarities for the paragraphs.
#     queryEmbeddingList.append(paragraphEmbeddingList)
#     querySimilarityList.append(paragraphSimilarity)
#     end_time = time.time() - start
#     print(end_time)
    
# with open("../Java_API/Car-Grpc/src/main/resources/data/Local_ELMo/embeddingsView.txt", "w") as e:
#     e.write("{" + str(querySimilarityList) + "}")

In [ ]:
# for query in data.get("queries")[:10]: 
#     queryFormatted = list(set(formatQuery(query.get("QueryId"),False).replace("enwiki:","").split(" ")))
#     averageQueryEmbedding = np.average(elmo.embed_sentence(queryFormatted),axis=0)
#     print(averageQueryEmbedding.shape)
#     for paragraph in query.get("Paragraphs")[:10]:
#         paragraph = paragraph.replace("[","").replace("]","")
        
#         test = ' '.join(paragraph.split(","))
#         sentences = test.split("fullstop")
#         for x in range(0,len(sentences)):
#             cleanedSentence = sentences[x].replace("  "," ").strip().split(" ")
#             sentences[x] = cleanedSentence
#             # Input query as a list
#         sentencesEmbeddings = elmo.embed_batch(sentences)
#         for sentenceEmbedding in sentencesEmbeddings:
#             perWordEmbedding = np.average(sentenceEmbedding, axis=0)
#             similarity = dict()
#             for wordEmbed in perWordEmbedding:
#                 for qWordEmbed in averageQueryEmbedding:
#                     similarity[qWordEmbed]
#                     print(scipy.spatial.distance.cosine(wordEmbed,qWordEmbed))
# #         # Calculate the average embedding for each word
# #         for x in range(len(tokens)):
# #             avg_tokens.append((vectors[0][x] + vectors[1][x] + vectors[2][x])/3)
# #     print((x).get("QueryId"))

# # print(data.get("queries")[0].get("QueryId"))
# # print(data.get("queries")[0].get("Paragraphs"))